In [1]:
import numpy as np
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import os
import cv2

In [ ]:
def image_test():
    #Bundle everything together
    prototextpath = os.path.sep.join(['face_detector', 'deploy.prototxt'])
    weightpath = os.path.sep.join(['face_detector', 'res10_300x300_ssd_iter_140000.caffemodel'])
    network=cv2.dnn.readNet(prototextpath,weightpath)
    model = load_model('model.h5')
    image = cv2.imread('test_images/facemask.jpg')
    h,w = image.shape[:2]
    blob = cv2.dnn.blobFromImage(image, 1.0, (300,300), (104.0, 177.0, 123.0))
    network.setInput(blob)
    detection = network.forward()
    for i in range(0,detection.shape[2]):
        confidence = detection[0,0,i,2]
        if confidence > 0.5:
            #Get the X and Y coordinates
            box=detection[0,0,i,3:7]*np.array([h,w,h,w])
            (startx, starty, endx, endy) = box.astype('int')

            #Ensure the bounding boxes fall within the dimensions of the frame
            startx, starty = (max(0,startx), max(0,starty))
            endx, endy = (min(w-1,endx), min(h-1, endy))

            #Extract the ROI of the image, convert it to RGB and resize it to 350x350 and preprocess it
            face = image[starty:endy, startx:endx]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (350,350))
            face = img_to_array(face)
            face = preprocess_input(face)
            face = np.expand_dims(face, axis=0)

            with_mask, without_mask = model.predict(face)[0]

            #determine the bounding box and label color
            label = 'Mask' if with_mask > without_mask else 'No mask'
            color = (0,255,0) if label =='Mask' else (0,0,255) 

            #include probability in the label
            label = '{} : {:.1f}%'.format(label, max(with_mask, without_mask)*100)

            #Display the label and bounding boxes
            cv2.putText(image, label, (startx,starty-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
            cv2.rectangle(image,(startx,starty), (endx,endy), color, 2)
    return cv2.imshow('Prediction',image)
        
image_test()
cv2.waitKey(0)
cv2.destroyAllWindows()